## Getting data from BigQuery



In [21]:
import pandas as pd
import os.path, os
import time
from google.cloud import bigquery,storage

pd.set_option('display.max_columns', None)


### Workflow for downloading files from BigQuery (via Google Cloud Storage)

#### Setup parameters in preparation for transferring files from BigQuery to Cloud Storage, and then to download them locally

In [20]:
bq_file_fo_path='./'
project_id = 'mitx-edx-data'
bucket_name = 'mitx-edx-data'

local_dl_folder = bq_file_fo_path

credentials = 'xxx.json'
###### THIS LINE ABOVE IS SUPER-SENSITIVE AND TAKE GOOD CARE TO PROTECT THIS FILE.

print(local_dl_folder)

./


### Below are a couple of utility functions that hopefully you don't have to deal with...

In [3]:
def extract_gbq_to_bucket(project_id,dataset,table_name,bucket_name,fo_path,credentials):
    client = bigquery.Client.from_service_account_json(credentials) #activate bigquery access
    
    json_tables = ['problem_analysis','course_axis']
    
    if table_name in json_tables:
        file_name = dataset+'-'+table_name+'.json'
    else:
        file_name = dataset+'-'+table_name+'.csv'
        
    print(file_name)
    destination_uri = 'gs://{}/{}/{}'.format(bucket_name, fo_path, file_name)
    dataset_ref = client.dataset(dataset, project=project_id)
    table_ref = dataset_ref.table(table_name)
    job_config = bigquery.job.ExtractJobConfig()
    print(job_config)
    if table_name in json_tables:
        job_config.destination_format = 'NEWLINE_DELIMITED_JSON'
    #job_config.dry_run = True   # Doesn't really work!
    
    extract_job = client.extract_table(table_ref, destination_uri, job_config=job_config)
    extract_job.result()  # Waits for job to complete.
    print(extract_job)
    print('Exported {}:{}.{} to {}'.format(project_id, dataset, table_name, destination_uri))
    return file_name


def download_gs_files_to_local(project_id,bucket_name,fo_path,file_name,local_dl_folder,credentials):
    
    gs_file_path = '{}/{}'.format(fo_path,file_name)
    local_file_path = '{}/{}'.format(local_dl_folder,file_name)
    # Initialise a client
    storage_client = storage.Client.from_service_account_json(credentials) #activate bigquery access
    # Create a bucket object for our bucket
    bucket = storage_client.get_bucket(bucket_name)  #oh no! Buckets name doesn't contain the folder!
    # Create a blob object from the filepath
    blob = bucket.blob(gs_file_path) #Include the google storage folder names here.
    # Download the file to a destination
    blob.download_to_filename(local_file_path) #This is the local file path

### Define the datasets and tables that you want to download:

In [ ]:
#fo_path = 'MITx_residential_data' #this is in gcp
fo_path = 'temp_data_transfer' #this is in gcp
dataset_lists = ['MITx__8_02r_9__2019_Spring'
]
table_lists = [#'problem_analysis',
    ##'problem_check',
    'person_course',
    ##'problem_grades'
    #'person_course_day',
    #'time_on_task',
    #'course_axis',
    #'show_answer_stats_by_user',
    ##'stats_for_problems',
    #'video_axis',
    #'video_stats',
    #'video_stats_day',
    #'person_course_video_watched',
            ]

### Run the following cell to get stuff from BQ via GCS

There are 2 main segments in this: exporting from BQ to GCS and from GCS to local disk.

The following cell is good when you have a small number of tables to download. However, if you have a large number of files to download, the preferable methods is via gsutil.

https://cloud.google.com/storage/docs/quickstart-gsutil

'''
To do so, once the tables are exported from BigQuery to Google Storage, use gsutil in command line to
download them to the local folder. First, change your directory to where you want to be, and then type:
gsutil -m cp gs://mitx-edx-data/moocs/* ./

m -> is the batch processing command which goes much faster than working interactively
'''

In [8]:
#Get data: GBQ --> Google Cloud Storage --> local disk
for dataset in dataset_lists:
    for table_name in table_lists:
        try:
            #The following line exports the table from BigQuery to Google Cloud Storage
            file_name = extract_gbq_to_bucket(project_id,dataset,table_name,bucket_name,fo_path, credentials)
            
            #The following line downloads the file from Google Cloud Storage to local folder
            download_gs_files_to_local(project_id,bucket_name,fo_path,file_name,local_dl_folder, credentials)
            time.sleep(1) #the download job takes time
            pass
        except Exception as e:
            print('Failed to download {} due to error {}'.format(table_name, e))
        
#wc -l [filenames*] lets you see the line numbers in each file, that makes it easy to check manually against
#the table details on BQ to make sure that all the data got downloaded. For a large number of files this step
#has to be automated. Still TO DO.

MITx__8_02r_9__2019_Spring-person_course.csv
Exported mitx-edx-data:MITx__8_02r_9__2019_Spring.person_course to gs://mitx-edx-data/temp_data_transfer/MITx__8_02r_9__2019_Spring-person_course.csv


### Get the list of datasets, tables, etc.

These are non essential, but useful commands...

In [11]:
client = bigquery.Client.from_service_account_json(credentials) 
datasets = list(client.list_datasets())


for dataset in datasets[:3]:  # API request(s)
    print(dataset.dataset_id)
    


2_S989r__2019_IAP__2019_IAP
2_S989r__2019_IAP__2019_IAP_logs
MITProfessionalX__6_BDX__2015_T3_latest


In [19]:
dataset_ref = {
    "datasetId": "2_S989r__2019_IAP__2019_IAP", 
    "projectId": "mitx-edx-data"
  }

tables = list(client.list_tables(bigquery.dataset.DatasetReference(dataset_ref['projectId'], 
                                                                   dataset_ref['datasetId'])))
for table_ in tables[:5]:
    print(table_, table_.table_id, table_.__dict__)
    #or
    print(table_._properties['tableReference']['tableId'])

<google.cloud.bigquery.table.TableListItem object at 0x10eadfe48> problem_analysis {'_properties': {'kind': 'bigquery#table', 'id': 'mitx-edx-data:2_S989r__2019_IAP__2019_IAP.problem_analysis', 'tableReference': {'projectId': 'mitx-edx-data', 'datasetId': '2_S989r__2019_IAP__2019_IAP', 'tableId': 'problem_analysis'}, 'type': 'TABLE', 'creationTime': '1550860788859'}}
problem_analysis
<google.cloud.bigquery.table.TableListItem object at 0x10eadfc18> problem_grades {'_properties': {'kind': 'bigquery#table', 'id': 'mitx-edx-data:2_S989r__2019_IAP__2019_IAP.problem_grades', 'tableReference': {'projectId': 'mitx-edx-data', 'datasetId': '2_S989r__2019_IAP__2019_IAP', 'tableId': 'problem_grades'}, 'type': 'TABLE', 'creationTime': '1550860756864'}}
problem_grades
<google.cloud.bigquery.table.TableListItem object at 0x10eb31eb8> studentmodule {'_properties': {'kind': 'bigquery#table', 'id': 'mitx-edx-data:2_S989r__2019_IAP__2019_IAP.studentmodule', 'tableReference': {'projectId': 'mitx-edx-data